# First Class Patterns

In [1]:
#require "core"
open Core

/Users/theowang/.opam/4.12.0/lib/base/base_internalhash_types: added to search path
/Users/theowang/.opam/4.12.0/lib/base/base_internalhash_types/base_internalhash_types.cma: loaded
/Users/theowang/.opam/4.12.0/lib/base/caml: added to search path
/Users/theowang/.opam/4.12.0/lib/base/caml/caml.cma: loaded
/Users/theowang/.opam/4.12.0/lib/base/shadow_stdlib: added to search path
/Users/theowang/.opam/4.12.0/lib/base/shadow_stdlib/shadow_stdlib.cma: loaded
/Users/theowang/.opam/4.12.0/lib/sexplib0: added to search path
/Users/theowang/.opam/4.12.0/lib/sexplib0/sexplib0.cma: loaded
/Users/theowang/.opam/4.12.0/lib/base: added to search path
/Users/theowang/.opam/4.12.0/lib/base/base.cma: loaded
/Users/theowang/.opam/4.12.0/lib/base/md5: added to search path
/Users/theowang/.opam/4.12.0/lib/base/md5/md5_lib.cma: loaded
/Users/theowang/.opam/4.12.0/lib/fieldslib: added to search path
/Users/theowang/.opam/4.12.0/lib/fieldslib/fieldslib.cma: loaded
/Users/theowang/.opam/4.12.0/lib/ppx_compar

In [2]:
type (_, _, _) pat = 
    Any : ('a, 'r, 'r) pat
  | Int : int -> (int, 'r, 'r) pat
  | Var : ('a, 'a -> 'r, 'r) pat (* Would be 'a code here instead *)
  | EmptyList : ('a list, 'r, 'r) pat
  | Pair : ('a, 'k, 'j) pat * ('b, 'j, 'r) pat -> ('a * 'b, 'k, 'r) pat
  | Cons : ('a, 'k, 'j) pat * ('a list, 'j, 'r) pat -> ('a list, 'k, 'r) pat

type (_, _, _) pat =
    Any : ('a, 'r, 'r) pat
  | Int : int -> (int, 'r, 'r) pat
  | Var : ('a, 'a -> 'r, 'r) pat
  | EmptyList : ('a list, 'r, 'r) pat
  | Pair : ('a, 'k, 'j) pat * ('b, 'j, 'r) pat -> ('a * 'b, 'k, 'r) pat
  | Cons : ('a, 'k, 'j) pat * ('a list, 'j, 'r) pat -> ('a list, 'k, 'r) pat


In [3]:
let rec match_ : type a f r. a -> (a, f, r) pat -> f -> r = (* Explicit type notation *)
  fun scrutinee pattern cont -> 
    match pattern, scrutinee with
    | Any, _ -> cont
    | Int i, _ -> if i = scrutinee then cont else failwith "match failed"
    | Var, _ -> cont scrutinee
    | EmptyList, _ -> cont
    | Pair (p1, p2), (a1, a2) -> 
      let m1 = match_ a1 p1 cont in
      let m2 = match_ a2 p2 m1 in
      m2
    | Cons (p1, p2), a1::a2 -> 
        let m1 = match_ a1 p1 cont in
        let m2 = match_ a2 p2 m1 in
        m2
    | _, _ -> failwith "match failed"

val match_ : 'a -> ('a, 'f, 'r) pat -> 'f -> 'r = <fun>


# Tagless final embedding

In [4]:
module type Symantics = sig
  type ('c, 'dv) repr
  val int : int  -> ('c, int) repr
  val bool: bool -> ('c, bool) repr
  val lam : (('c, 'da) repr -> ('c, 'db) repr) -> ('c, 'da -> 'db) repr
  val app : ('c, 'da -> 'db) repr -> ('c, 'da) repr -> ('c, 'db) repr
  val fix : ('x -> 'x) -> (('c, 'da -> 'db) repr as 'x)
  val add : ('c, int) repr -> ('c, int) repr -> ('c, int) repr
  val mul : ('c, int) repr -> ('c, int) repr -> ('c, int) repr
  val leq : ('c, int) repr -> ('c, int) repr -> ('c, bool) repr
  val if_ : ('c, bool) repr -> (unit -> 'x) -> (unit -> 'x) -> (('c, 'da) repr as 'x)
end

module type Symantics =
  sig
    type ('c, 'dv) repr
    val int : int -> ('c, int) repr
    val bool : bool -> ('c, bool) repr
    val lam : (('c, 'da) repr -> ('c, 'db) repr) -> ('c, 'da -> 'db) repr
    val app : ('c, 'da -> 'db) repr -> ('c, 'da) repr -> ('c, 'db) repr
    val fix :
      (('c, 'da -> 'db) repr -> ('c, 'da -> 'db) repr) ->
      ('c, 'da -> 'db) repr
    val add : ('c, int) repr -> ('c, int) repr -> ('c, int) repr
    val mul : ('c, int) repr -> ('c, int) repr -> ('c, int) repr
    val leq : ('c, int) repr -> ('c, int) repr -> ('c, bool) repr
    val if_ :
      ('c, bool) repr ->
      (unit -> ('c, 'da) repr) -> (unit -> ('c, 'da) repr) -> ('c, 'da) repr
  end


In [5]:
(* https://okmij.org/ftp/tagless-final/JFP.pdf *)
module Interpreter : Symantics = struct
  type ('c, 'dv) repr = 'dv
  let int x = x
  let bool b = b
  let lam f = f
  let app e1 e2 = e1 e2 
  let fix f = let rec self n = f self n in self
  let add e1 e2 = e1 + e2
  let mul e1 e2 = e1 * e2
  let leq e1 e2 = e1 <= e2
  let if_ eb et ee = if eb then et () else ee ()
end

module Interpreter : Symantics


In [6]:
let module I = Interpreter in I.app (I.lam (fun x -> x)) (I.bool true)

- : ('_weak1, bool) Interpreter.repr = <abstr>


In [7]:
module type Pat = sig
  type ('c, 'a, 'f, 'r) pat
  val int_ : int -> ('c, int, 'r, 'r) pat
  val any_ : ('c, 'a, 'r, 'r) pat
  val var : ('c, 'a, 'a -> 'r, 'r) pat
  val emptylist : ('c, 'a list, 'r, 'r) pat
  val pair : ('c, 'a, 'k, 'j) pat -> ('c, 'b, 'j, 'r) pat -> ('c, 'a * 'b, 'k, 'r) pat
  val cons : ('c, 'a, 'k, 'j) pat -> ('c, 'a list, 'j, 'r) pat -> ('c, 'a list, 'k, 'r) pat
  val or_ : ('c, 'a, 'k, 'r) pat -> ('c, 'a, 'k, 'r) pat -> ('c, 'a, 'k, 'r) pat
end

module type Pat =
  sig
    type ('c, 'a, 'f, 'r) pat
    val int_ : int -> ('c, int, 'r, 'r) pat
    val any_ : ('c, 'a, 'r, 'r) pat
    val var : ('c, 'a, 'a -> 'r, 'r) pat
    val emptylist : ('c, 'a list, 'r, 'r) pat
    val pair :
      ('c, 'a, 'k, 'j) pat ->
      ('c, 'b, 'j, 'r) pat -> ('c, 'a * 'b, 'k, 'r) pat
    val cons :
      ('c, 'a, 'k, 'j) pat ->
      ('c, 'a list, 'j, 'r) pat -> ('c, 'a list, 'k, 'r) pat
    val or_ :
      ('c, 'a, 'k, 'r) pat -> ('c, 'a, 'k, 'r) pat -> ('c, 'a, 'k, 'r) pat
  end


In [8]:
module PattnMatch = struct
  open Core.Option.Monad_infix
  type ('c, 'a, 'f, 'r) pat = 'a -> 'f -> 'r option
  let int_ i scrutinee cont = if i = scrutinee then Some (cont) else None
  let any_ scrutinee cont = Some cont
  let var scrutinee cont = Some (cont scrutinee)
  let emptylist scrutinee cont = Some cont
  let pair constr_left constr_right (scrutinee_left, scrutinee_right) cont = 
    constr_left scrutinee_left cont >>= fun m1 -> 
    constr_right scrutinee_right m1 >>= fun m2 -> 
    Some m2
  let cons constr_left constr_right (scrutinee) cont = 
    match scrutinee with 
    | scrutinee_left :: scrutinee_right ->
      constr_left scrutinee_left cont >>= fun m1 -> 
      constr_right scrutinee_right m1 >>= fun m2 ->
      Some m2
    | _ -> None
  let or_ constr_left constr_right scrutinee cont = 
      match constr_left scrutinee cont with
      | Some res -> Some res
      | None -> constr_right scrutinee cont
  let rename constr scrutinee cont = constr scrutinee (fun x y -> cont y x)
  (* TODO: implement this sort of renaming *)
end

module PattnMatch :
  sig
    type ('c, 'a, 'f, 'r) pat = 'a -> 'f -> 'r option
    val int_ : Core.Int.t -> Core.Int.t -> 'a -> 'a option
    val any_ : 'a -> 'b -> 'b option
    val var : 'a -> ('a -> 'b) -> 'b option
    val emptylist : 'a -> 'b -> 'b option
    val pair :
      ('a -> 'b -> 'c Base.Option.t) ->
      ('d -> 'c -> 'e Base.Option.t) -> 'a * 'd -> 'b -> 'e Base.Option.t
    val cons :
      ('a -> 'b -> 'c Base.Option.t) ->
      ('a list -> 'c -> 'd Base.Option.t) ->
      'a list -> 'b -> 'd Base.Option.t
    val or_ :
      ('a -> 'b -> 'c option) ->
      ('a -> 'b -> 'c option) -> 'a -> 'b -> 'c option
    val rename :
      ('a -> ('b -> 'c -> 'd) -> 'e) -> 'a -> ('c -> 'b -> 'd) -> 'e
  end


In [9]:
PattnMatch.pair PattnMatch.var PattnMatch.var (1, 2) (fun x -> fun y -> x + y)

- : int Base.Option.t = Base.Option.Some 3


In [10]:
PattnMatch.or_ (PattnMatch.pair PattnMatch.var (PattnMatch.int_ (3))) (PattnMatch.pair (PattnMatch.int_ (3)) PattnMatch.var) (3, 2) (fun x -> x)

- : Core.Int.t Base.Option.t = Base.Option.Some 2


In [11]:
PattnMatch.pair PattnMatch.var PattnMatch.var

- : '_weak2 * '_weak3 ->
    ('_weak2 -> '_weak3 -> '_weak4) -> '_weak4 Base.Option.t
= <fun>


In [12]:
PattnMatch.pair (PattnMatch.int_ (3)) PattnMatch.var

- : Core.Int.t * '_weak5 -> ('_weak5 -> '_weak6) -> '_weak6 Base.Option.t =
<fun>


In [13]:
PattnMatch.pair (PattnMatch.int_ (3)) PattnMatch.var (3, 2) (fun x -> x)

- : int Base.Option.t = Base.Option.Some 2


# Tagless Final, but with DB indices

Need to try to see if I can generate things with De Bruijn indices instead, and what it could look like. Current issue is that we're encoding the context $\Gamma$ as this curried function `'f`, and this is similar in spirit to doing HOAS. The issue, of course, is that I'd like a more explicit definition of variables so that I can manipulate them and rename them etc.

In [ ]:
(x, y) \intersect (y, x) -> 
  (x, y)

  (x, y) -| x: int -> y: int -> 'r
  (y, x) -| x: int -> y: int -> 'r

  (2, 2)